In [180]:
import pandas as pd

#steve-report is a recording of my shifts to-date. routes.csv is all routes from Google GTFS
driver_report = pd.read_csv('steve-report.csv')
routes = pd.read_csv('routes.csv')
#print(routes)
print(driver_report)

            shiftname start time  end time  length        date  route  \
0           UCREW3EVE      19:30     21:30    2.00  2014-01-20    NaN   
1                  30   18:30:00  21:30:00    3.00  2014-01-24    NaN   
2              UCREW4   18:30:00  21:30:00    3.00  2014-01-27    NaN   
3             Trainee      10:02     11:27    1.50  2014-01-13    NaN   
4                Fees      12:00     17:45    5.75  2014-01-12    NaN   
...               ...        ...       ...     ...         ...    ...   
1363  31-3 PM1 Part 2      15:00     15:06    0.00  2018-05-08   31.0   
1364   33-2 AM Part 1       7:21      8:51    1.50  2018-05-10   33.0   
1365     Service Work       6:59      8:34    1.50  2018-05-25    NaN   
1366   36-1 AM Part 2       8:47     10:15    1.50  2018-06-08   36.0   
1367   36-1 AM Part 3      10:15     10:17    0.00  2018-06-08   36.0   

     department  
0       Service  
1       Service  
2       Service  
3      Training  
4      Training  
...         ...

In [187]:
#remove decimal and trailing zero from data (numpy and pandas require this)
driver_report['route'] = driver_report.route.astype(str)
driver_report['route'] = driver_report.route.str.replace('.0', ' ')

#print (driver_report)

#change the report column back to numbers so it can be compared to the other CSV:
driver_report["route"] = pd.to_numeric(driver_report["route"], errors='ignore')
#same with the Routes .csv:
routes["route_id"] = pd.to_numeric(routes["route_id"], errors='ignore')


/var/folders/gt/yj438g6s03dchf7qmhfj45640000gn/T/ipykernel_42498/1005083215.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  driver_report['route'] = driver_report.route.str.replace('.0', ' ')


In [190]:
#these work for exact match but not fuzzy match
#print(routes[routes.route_id.isin(driver_report.route)])
#print(driver_report[driver_report.route.isin(routes.route_id)])

#pre-2014 legacy bus shift naming convention not captured here.

am_shifts = '^[0-9]{2}-[0-9] AM(.*)$'
mid_shifts = '^[0-9]{2}-[0-9] MID(.*)$'
pm_shifts = '^[0-9]{2}-[0-9] PM(.*)$'
pm2_shifts = '^[0-9]{2}-[0-9] PM2(.*)$'
eve_shifts = '^[0-9]{2}-[0-9] EVE(.*)$'

#this gives me a rudimentary way of seeing what shifts I did in morning, afternoon, and evening blocks. For example:
#print(driver_report[driver_report.shiftname.str.contains(mid_shifts)])

driver_report[driver_report.shiftname.str.contains(mid_shifts)]

#I can join these regexes together and see all shifts:
#allshifts = [am_shifts, mid_shifts, pm_shifts, pm2_shifts, eve_shifts]
#pattern = ('|'.join(allshifts))
#print(driver_report[driver_report.shiftname.str.contains(pattern)])

#I can also just sort on "bus"



/var/folders/gt/yj438g6s03dchf7qmhfj45640000gn/T/ipykernel_42498/4027447413.py:16: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  driver_report[driver_report.shiftname.str.contains(mid_shifts)]


,shiftname,start time,end time,length,date,route,department


In [207]:
#Now I want to analyze the data.

#average length of a MID shift
#driver_report.length[driver_report.shiftname.str.contains(mid_shifts)].describe().mean



/var/folders/gt/yj438g6s03dchf7qmhfj45640000gn/T/ipykernel_42498/4160004002.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  driver_report.length[driver_report.shiftname.str.contains(eve_shifts)].describe().max


<bound method NDFrame._add_numeric_operations.<locals>.max of count    45.000000
mean      4.766667
std       1.109617
min       1.250000
25%       4.250000
50%       4.750000
75%       5.750000
max       7.250000
Name: length, dtype: float64>